In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns   
from pathlib import Path






In [ ]:

data_path ="../data/raw/telecom_customer_churn.csv"

df = pd.read_csv(data_path)
df.head()


In [ ]:
df.shape

In [ ]:
df["Churn"] = df["Customer Status"].apply(lambda x: 1 if x == "Churned" else 0)
df.head()

Columns to drop:

Customer ID
Customer Status
Churn Category
Churn Reason

categorical columns
| Column                                           | Missing       | Strategy           |
| ------------------------------------------------ | ------------- | ------------------ |
| Offer                                            | ~3877 missing | Fill "No Offer"    |
| Multiple Lines                                   | ~682 missing  | Fill "No Phone"    |
| Internet Type                                    | ~1526 missing | Fill "No Internet" |
| Internet-related features (Online Security etc.) | ~1526 missing | Fill "No Internet" |


numerical columns


| Avg Monthly Long Distance Charges                | ~682 missing  | Fill 0 if no phone |

| Avg Monthly GB Download                          | ~1526 missing | Fill 0             |

In [ ]:
df.drop(columns=["Customer ID", "Customer Status", "Churn Category","Churn Reason"], inplace=True)
df.head()

In [ ]:
df["Avg Monthly Long Distance Charges"]

In [ ]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    print(f"{column}: {len(outliers)} outliers")
    return outliers


In [ ]:
detect_outliers_iqr(df, "Avg Monthly Long Distance Charges")

In [ ]:
detect_outliers_iqr(df, "Avg Monthly GB Download")


In [ ]:
sns.histplot(df["Avg Monthly GB Download"], kde=True)
plt.show()

In [ ]:
df[df["Phone Service"] == "No"].shape


we are filling null values in Avg Monthly Long Distance Charges=0
Reason: If no phone service → no long distance → charges should be 0.

In [ ]:
df["Avg Monthly Long Distance Charges"].fillna(0, inplace=True)

In [ ]:
df[df["Internet Service"] == "No"].shape


In [ ]:
df["Avg Monthly GB Download"].fillna(0, inplace=True)

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=["City", "Zip Code", "Latitude", "Longitude"])


Offer → 3166 non-null

Multiple Lines → 6361 non-null

 13  Online Security                    5517 non-null   object 

 14  Online Backup                      5517 non-null   object  
 
 15  Device Protection Plan             5517 non-null   object  
 
 16  Premium Tech Support               5517 non-null   object  
 
 17  Streaming TV                       5517 non-null   object  
 
 18  Streaming Movies                   5517 non-null   object  
 
 19  Streaming Music                    5517 non-null   object 


Usage_Segment → 5517 non-null


In [ ]:
df["Offer"] = df["Offer"].fillna("No Offer")
df["Multiple Lines"] = df["Multiple Lines"].fillna("No Phone")


In [ ]:
df["Internet Type"] = df["Internet Type"].fillna("No Internet Service")
df["Internet Type"] = df["Internet Type"].fillna("No Internet Service")


In [ ]:
internet_cols = [
    "Online Security",
    "Online Backup",
    "Device Protection Plan",
    "Premium Tech Support",
    "Streaming TV",
    "Streaming Movies",
    "Streaming Music",
    "Unlimited Data"
]

for col in internet_cols:
    df[col] = df[col].fillna("No Internet Service")


In [ ]:
df["Offer"].value_counts()

In [ ]:
df["Multiple Lines"].isnull().sum()

In [ ]:
df["Multiple Lines"].value_counts()

In [ ]:
df.info()

In [ ]:
df["Internet Service"].value_counts()

In [ ]:
df["Internet Type"].value_counts()

In [ ]:
# df["Usage_Segment"] = pd.cut(
#     df["Avg Monthly GB Download"],
#     bins=[0, 30, 60, df["Avg Monthly GB Download"].max()],
#     labels=["Low", "Medium", "High"]
# )
# df["Usage_Segment"].value_counts()

In [ ]:
df["Streaming Music"].value_counts()

In [ ]:
df["Streaming Movies"].value_counts()

In [ ]:
df["Contract"].value_counts()

In [ ]:
df["Total Revenue"].min()

In [ ]:
df["Tenure in Months"].min()


In [ ]:
df["Streaming TV"].value_counts()

FEATURE ENGINEERING

In [ ]:
df["Revenue_Per_Month"] = df["Total Revenue"] / df["Tenure in Months"]
df["Is_Monthly_Contract"] = (df["Contract"] == "Month-to-Month").astype(int)



In [ ]:
df["Unlimited Data"].value_counts()

In [ ]:
df.columns

In [ ]:

df.groupby("Churn")["Is_Monthly_Contract"].mean()

In [ ]:
df.groupby("Churn")["Revenue_Per_Month"].mean()

In [ ]:
pd.crosstab(df["Streaming TV"], df["Churn"], normalize="index")


In [ ]:
df.info()

ONE HOT ENCODING

In [ ]:
categ_col= df.select_dtypes(include=["object"]).columns
categ_col

In [ ]:
binary_map = {
    "Gender": {"Male": 1, "Female": 0},
    "Married": {"Yes": 1, "No": 0},
    "Phone Service": {"Yes": 1, "No": 0},
    "Paperless Billing": {"Yes": 1, "No": 0},
    "Internet Service": {"Yes": 1, "No": 0}
}

for col, mapping in binary_map.items():
    df[col] = df[col].map(mapping)
df.head()

In [ ]:
dummy_cols = [
    "Offer",
    "Multiple Lines",
    "Internet Type",
    "Online Security",
    "Online Backup",
    "Device Protection Plan",
    "Premium Tech Support",
    "Streaming TV",
    "Streaming Movies",
    "Streaming Music",
    "Unlimited Data",
    "Payment Method"
]

df = pd.get_dummies(df, columns=dummy_cols, drop_first=True)
df.head()

In [ ]:
df = df.drop(columns=["Contract"])
df.head()

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler

In [ ]:
x=df.drop(columns=["Churn"])
y=df["Churn"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

model training 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model=LogisticRegression()
model.fit(x_train_scaled, y_train)

In [ ]:
y_pred = model.predict(x_test_scaled)



In [ ]:
y_pred

In [ ]:
y_probs = model.predict_proba(x_test_scaled)[:, 1]


In [ ]:
y_probs

In [ ]:
print(y_probs[:20])
print(min(y_probs), max(y_probs))


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, y_probs)

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

for t in thresholds:
    y_pred_custom = (y_probs >= t).astype(int)
    print(f"\nThreshold: {t}")
    print(classification_report(y_test, y_pred_custom))

In [ ]:
print(classification_report(y_test, y_pred))


In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
print("Train Accuracy:", model.score(x_train, y_train))
print("Test Accuracy:", model.score(x_test, y_test))

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

DECIISON TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(
        max_depth=8,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        class_weight="balanced"
    )
    
tree.fit(x_train_scaled, y_train)
y_pred = tree.predict(x_test_scaled)
print(roc_auc_score(y_test, tree.predict_proba(x_test_scaled)[:, 1]))

In [ ]:
depths = range(2, 21)

for d in depths:
    tree = DecisionTreeClassifier(
        max_depth=d,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42
    )
    
    tree.fit(x_train_scaled, y_train)
    y_pred = tree.predict(x_test_scaled)
    
    classification_report_Data = classification_report(y_test, y_pred)
    
    print(f"Depth: {d} | Train: {tree.score(x_train_scaled, y_train):.3f} | Test: {tree.score(x_test_scaled, y_test):.3f} ")
    print(classification_report_Data)

In [ ]:
tree.fit(x_train, y_train)

In [ ]:
y_pred = tree.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print("Train Accuracy:", tree.score(x_train, y_train))
print("Test Accuracy:", tree.score(x_test, y_test))